In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
import requests

In [9]:
# Chave da API do Alpha Vantage
ALPHA_VANTAGE_API_KEY = 'ITHZKPVGZ2QKUXW0'

In [15]:
# Função para baixar dados históricos de uma ação usando Alpha Vantage API com requests
def download_data_av(symbol, interval='5min'):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={ALPHA_VANTAGE_API_KEY}'
    response = requests.get(url)
    data = response.json()

    # Extrair dados do dicionário JSON para DataFrame pandas
    if 'Time Series (' in data:
        time_series_data = data['Time Series (' + interval + ')']
        df = pd.DataFrame(time_series_data).transpose()
        df.index = pd.to_datetime(df.index)
        df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        df = df.astype(float)
        print(df)
        return df
    else:
        print("Erro ao obter dados do Alpha Vantage.")
        return None

In [11]:
# Função para preparar os dados
def prepare_data(data):
    data_selected = data[['Open', 'Close', 'Volume']]
    data_selected['Average'] = data_selected[['Open', 'Close']].mean(axis=1)
    return data_selected[['Volume', 'Average']]

In [12]:
# Função para criar as matrizes e os labels
def create_matrices(data_final):
    num_rows = data_final.shape[0]
    num_matrices = num_rows // 5  # Número de matrizes completas que podemos formar

    matrices = []
    matrices_y = []

    for i in range(num_matrices):
        start_idx = i * 5
        end_idx = start_idx + 5
        matrix = data_final.iloc[start_idx:end_idx].copy()

        # Calcular a soma da coluna de volume
        volume_sum = matrix['Volume'].sum()

        # Tornar a coluna do volume relativa
        matrix['Volume'] = matrix['Volume'] / volume_sum

        # Converter para valores numpy e adicionar à lista de matrizes
        matrices.append(matrix.values)

        # Criar labels
        if i > 0:  # Ignorar a primeira matriz para alinhamento correto
            first_value = matrices[i][0, 1]
            last_value = matrices[i][4, 1]
            delta = last_value - first_value
            delta_bi = 1 if delta > 0 else 0
            matrices_y.append(delta_bi)

    # Ordenar as linhas de cada matriz por ordem decrescente da coluna Average (segunda coluna)
    matrices_x = []
    for matrix in matrices:
        sorted_indices = np.argsort(matrix[:, 1])[::-1]
        sorted_matrix = matrix[sorted_indices]
        matrices_x.append(sorted_matrix)

    # Remover a última matriz de matrices_x para alinhamento com matrices_y
    if matrices_x:
        matrices_x.pop()

    # Converter listas para arrays numpy
    matrices_x = np.array(matrices_x)
    matrices_y = np.array(matrices_y)

    return matrices_x, matrices_y

In [13]:
# Função para construir a rede neural
def build_model(input_shape):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [16]:
# Função principal
def main():
    # Definir o ticker da ação
    ticker = 'AAPL'

    # Baixar dados históricos usando Alpha Vantage com requests
    data = download_data_av(ticker)

    if data is not None:
        # Preparar os dados
        data_final = prepare_data(data)

        # Criar as matrizes e os labels
        matrices_x, matrices_y = create_matrices(data_final)

        # Imprimir o tamanho do dataset
        print(f"Tamanho do dataset: {matrices_x.shape[0]}")

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(matrices_x, matrices_y, test_size=0.2, random_state=42)

        # Construir a rede neural
        model = build_model(input_shape=(5, 2))

        # Treinar o modelo
        model.fit(X_train, y_train, epochs=10, validation_split=0.2)

        # Avaliar o modelo
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f'Acurácia no conjunto de teste: {accuracy:.4f}')

        # Fazer previsões
        predictions = model.predict(X_test)
        predictions = (predictions > 0.5).astype(int)

        # Verificar o tamanho do conjunto de teste
        num_to_display = min(10, len(y_test))

        # Exibir as primeiras previsões e seus valores reais correspondentes
        for i in range(num_to_display):
            print(f'Predição: {predictions[i]}, Real: {y_test[i]}')
    else:
        print("Não foi possível baixar os dados do Alpha Vantage.")

if __name__ == "__main__":
    main()

Erro ao obter dados do Alpha Vantage.
Não foi possível baixar os dados do Alpha Vantage.
